In [1]:
import numpy as np
from pymatgen.io.cif import CifParser
from pymatgen.core import Structure
from pymatgen.core.lattice import Lattice

In [2]:
import sys
sys.path.append('../..')
from ai4mat.data.data import (
    StorageResolver)

In [3]:
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor

In [4]:
mp_cell = CifParser(StorageResolver()["materials"] / 
"V2O5_mp-25279_computed.cif").get_structures(primitive=False)[0]

In [5]:
mp_cell

Structure Summary
Lattice
    abc : 3.618658 4.797293 11.551508
 angles : 90.0 90.0 90.0
 volume : 200.53143762391144
      A : 3.618658 0.0 2.2157889684544813e-16
      B : -2.9374947585110014e-16 4.797293 2.9374947585110014e-16
      C : 0.0 0.0 11.551508
    pbc : True True True
PeriodicSite: V (-0.0000, 2.9347, 7.5027) [0.0000, 0.6117, 0.6495]
PeriodicSite: V (1.8093, 1.8626, 9.8246) [0.5000, 0.3883, 0.8505]
PeriodicSite: V (1.8093, 1.8626, 1.7269) [0.5000, 0.3883, 0.1495]
PeriodicSite: V (-0.0000, 2.9347, 4.0488) [0.0000, 0.6117, 0.3505]
PeriodicSite: O (1.8093, 2.4549, 7.8762) [0.5000, 0.5117, 0.6818]
PeriodicSite: O (-0.0000, 2.3424, 9.4510) [0.0000, 0.4883, 0.8182]
PeriodicSite: O (-0.0000, 2.3424, 2.1005) [0.0000, 0.4883, 0.1818]
PeriodicSite: O (1.8093, 2.4549, 3.6753) [0.5000, 0.5117, 0.3182]
PeriodicSite: O (-0.0000, 4.5437, 4.1005) [0.0000, 0.9471, 0.3550]
PeriodicSite: O (1.8093, 0.2536, 1.6752) [0.5000, 0.0529, 0.1450]
PeriodicSite: O (1.8093, 0.2536, 9.8763) [0.5000, 0.

In [6]:
view(AseAtomsAdaptor.get_atoms(mp_cell), viewer='ngl')

In [7]:
mp_cell.lattice.matrix

array([[ 3.61865800e+00,  0.00000000e+00,  2.21578897e-16],
       [-2.93749476e-16,  4.79729300e+00,  2.93749476e-16],
       [ 0.00000000e+00,  0.00000000e+00,  1.15515080e+01]])

In [8]:
lattice_np = np.array(mp_cell.lattice.matrix)
lattice_np[1, 1] = lattice_np[2, 2]
lattice_np[2, 2] = 20
tall_lattice = Lattice(lattice_np)

coords = np.array(mp_cell.cart_coords)
coords[:, [1, 2]] = coords[:, [2, 1]]

reference_cell = Structure(
    lattice=tall_lattice,
    species=mp_cell.species,
    coords=coords,
    coords_are_cartesian=True,
)

from pymatgen.io.cif import CifWriter
CifWriter(reference_cell).write_file(StorageResolver()["materials"] /"V2O5.cif")

In [9]:
reference_cell

Structure Summary
Lattice
    abc : 3.618658 11.551508 20.0
 angles : 90.0 90.0 90.0
 volume : 836.01913672528
      A : 3.618658 0.0 2.2157889684544813e-16
      B : -2.9374947585110014e-16 11.551508 2.9374947585110014e-16
      C : 0.0 0.0 20.0
    pbc : True True True
PeriodicSite: V (-0.0000, 7.5027, 2.9347) [0.0000, 0.6495, 0.1467]
PeriodicSite: V (1.8093, 9.8246, 1.8626) [0.5000, 0.8505, 0.0931]
PeriodicSite: V (1.8093, 1.7269, 1.8626) [0.5000, 0.1495, 0.0931]
PeriodicSite: V (-0.0000, 4.0488, 2.9347) [-0.0000, 0.3505, 0.1467]
PeriodicSite: O (1.8093, 7.8762, 2.4549) [0.5000, 0.6818, 0.1227]
PeriodicSite: O (-0.0000, 9.4510, 2.3424) [0.0000, 0.8182, 0.1171]
PeriodicSite: O (-0.0000, 2.1005, 2.3424) [-0.0000, 0.1818, 0.1171]
PeriodicSite: O (1.8093, 3.6753, 2.4549) [0.5000, 0.3182, 0.1227]
PeriodicSite: O (-0.0000, 4.1005, 4.5437) [-0.0000, 0.3550, 0.2272]
PeriodicSite: O (1.8093, 1.6752, 0.2536) [0.5000, 0.1450, 0.0127]
PeriodicSite: O (1.8093, 9.8763, 0.2536) [0.5000, 0.8550, 0.

In [11]:
supercell = reference_cell.copy()
supercell.make_supercell([6, 2, 1])
view(AseAtomsAdaptor.get_atoms(supercell), viewer='ngl')